## Follow up with Azure OpenAI the course ChatGPT Prompt Engineering for Developers

### Inferring

This notebook is based on the course: [chatgpt prompt eng: Inferring](https://learn.deeplearning.ai/chatgpt-prompt-eng/lesson/5/inferring)

In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('AZURE_OPENAI_KEY')
openai.api_type = "azure"
openai.api_version = "2023-08-01-preview"
openai.api_base = "https://cog-moww6huavlklg.openai.azure.com/"


In [30]:
def get_completion(prompt, model="gpt-3.5-turbo",deployment="chat", showAllResponse=False):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        engine=deployment,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    if showAllResponse:
        print(response)
    return response.choices[0].message["content"]

## Product review text

In [4]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

## Sentiment (positive/negative)

In [5]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt,showAllResponse=True)
print(response)

{
  "choices": [
    {
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      },
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "The sentiment of the product review is positive.",
        "role": "assistant"
      }
    }
  ],
  "created": 1704398881,
  "id": "chatcmpl-8dOIT0h7wa20vHUHvYmvrDGNYH2jI",
  "model": "gpt-35-turbo",
  "object": "chat.completion",
  "prompt_filter_results": [
    {
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "

In [6]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

positive


## Identify types of emotions

In [7]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

happy, satisfied, grateful, impressed, content


## Identify anger

In [8]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

No


## Extract product and company name from customer reviews

In [9]:
prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
  
Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
  "Item": "lamp",
  "Brand": "Lumina"
}


## Doing multiple tasks at once

In [10]:
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
  "Sentiment": "positive",
  "Anger": false,
  "Item": "lamp with additional storage",
  "Brand": "Lumina"
}


## Inferring topics

In [11]:
story = """
In a recent survey conducted by the government, 
public sector employees were asked to rate their level 
of satisfaction with the department they work at. 
The results revealed that NASA was the most popular 
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, 
stating, "I'm not surprised that NASA came out on top. 
It's a great place to work with amazing people and 
incredible opportunities. I'm proud to be a part of 
such an innovative organization."

The results were also welcomed by NASA's management team, 
with Director Tom Johnson stating, "We are thrilled to 
hear that our employees are satisfied with their work at NASA. 
We have a talented and dedicated team who work tirelessly 
to achieve our goals, and it's fantastic to see that their 
hard work is paying off."

The survey also revealed that the 
Social Security Administration had the lowest satisfaction 
rating, with only 45% of employees indicating they were 
satisfied with their job. The government has pledged to 
address the concerns raised by employees in the survey and 
work towards improving job satisfaction across all departments.
"""

## Infer 5 topics

In [12]:
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

government survey, job satisfaction, NASA, Social Security Administration, employee concerns


In [13]:
response.split(sep=',')

['government survey',
 ' job satisfaction',
 ' NASA',
 ' Social Security Administration',
 ' employee concerns']

In [14]:
topic_list = [
    "nasa", "local government", "engineering", 
    "employee satisfaction", "federal government"
]

## Make a news alert for certain topics

In [15]:
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple ticks.

Give your answer as list with 0 or 1 for each topic.\

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

nasa: 1
local government: 0
engineering: 0
employee satisfaction: 1
federal government: 1


### Experimentando con Datos propios

In [21]:
bauStory = f"""
BAU Dollar: Este beneficio permite a los colaboradores recibir una parte de su sueldo en dólares, siempre y cuando cumplan con ciertos requisitos. La retención de impuestos es la misma que para el sueldo en pesos.
BAU Lunch: Este beneficio ofrece a los colaboradores la posibilidad de recibir un monto mensual para utilizar en la compra de almuerzos. La periodicidad de las acreditaciones varía según el país. En algunos casos, es necesario registrarse en la App de PedidosYa para acceder al beneficio.
BAU Net: Este beneficio ofrece a los colaboradores un monto mensual para utilizar en el pago de servicios de internet. Es necesario presentar un comprobante de validez legal para acceder al beneficio.
BAU Health: Este beneficio ofrece a los colaboradores la posibilidad de recibir un reembolso por actividades físicas realizadas en un trimestre determinado. Es necesario presentar comprobantes de validez legal para acceder al beneficio.
Anti Tráfico: Este beneficio ofrece a los colaboradores la posibilidad de elegir un rango horario para evitar el tráfico en su traslado al trabajo. Se espera que una vez elegido un rango, se mantenga estable por al menos un mes.
BAU Lunch Latam: Este beneficio ofrece a los colaboradores la posibilidad de recibir un monto mensual para utilizar en la compra de almuerzos. Los valores de las acreditaciones varían según el país. Es necesario cumplir con ciertos requisitos para acceder al beneficio.
BAU Home Latam: Este beneficio ofrece a los colaboradores la posibilidad de recibir un monto mensual para utilizar en el pago de servicios de internet. Es necesario presentar un comprobante de validez legal para acceder al beneficio. Es importante tener en cuenta que algunos de estos beneficios tienen requisitos específicos para acceder a ellos y que es recomendable consultar periódicamente las tablas de comisiones o con el ejecutivo de cuenta para obtener información actualizada.
"""
bauTopicQuestion = ["Health", "Lunch", "Tráfico", "Anti Tráfico", "evitar tráfico"]


In [22]:
prompt = f"""
Determina cinco temas que están siendo discutidos en \
el siguiente texto que está delimitado por tres comillas \
simples.

Has cada tema de una o dos palabras de longitud.

Formatea tu respuesta como una lista de temas separados por comas.

Texto muestra: '''{bauStory}'''
"""
response = get_completion(prompt)
print(response)

BAU Dollar, BAU Lunch, BAU Net, BAU Health, Anti Tráfico.


In [23]:
prompt = f"""
Determine si cada elemento en la siguiente lista de \
temas es un tema en el texto siguiente, que
está delimitado por tres comillas simples.

Proporcione su respuesta como una lista con si o no en cada tema.

Lista de temas: {", ".join(bauTopicQuestion)}

Texto muestra: '''{bauStory}'''
"""
response = get_completion(prompt)
print(response)

Health - Sí
Lunch - Sí
Tráfico - No
Anti Tráfico - Sí
evitar tráfico - No


In [24]:
prompt = f"""
Determina los temas que están siendo discutidos en \
el siguiente texto que está delimitado por tres comillas \
simples.

Describe cada tema con una o dos palabras de longitud.

Formatea tu respuesta como una lista de temas separados por comas.

Texto muestra: '''{bauStory}'''
"""
response = get_completion(prompt)
print(response)

Beneficios laborales: BAU Dollar, BAU Lunch, BAU Net, BAU Health, Anti Tráfico, BAU Lunch Latam, BAU Home Latam.


In [25]:
prompt = f"""
Determina si cada elemento en la lista de \
temas es un tema en el texto que está delimitado \
por tres comillas simples.

Proporciona tu respuesta como una lista con si, no o parcialmente \
en cada tema que si coincida, no coincida o parcialmente coincida.

Lista de temas: {", ".join(bauTopicQuestion)}

Ficha técnica: '''{bauStory}'''
"""
response = get_completion(prompt)
print(response)

- Health: Sí
- Lunch: Sí
- Tráfico: No
- Anti Tráfico: Sí (parcialmente)
- evitar tráfico: No


In [26]:
prompt = f"""
Determina si cada elemento en la lista de \
temas es un tema en el texto que está delimitado \
por tres comillas simples.

Proporciona tu respuesta como una lista con si, no o parcialmente \
en cada tema que si coincida, no coincida o parcialmente coincida.

Si tu respuesta es parcialmente, justifica por que la coincidencia es parcial

Lista de temas: {", ".join(bauTopicQuestion)}

Ficha técnica: '''{bauStory}'''
"""
response = get_completion(prompt)
print(response)

Health: Sí (se menciona el beneficio de reembolso por actividades físicas)
Lunch: Sí (se mencionan dos beneficios relacionados con la compra de almuerzos)
Tráfico: No (no se menciona el tema del tráfico en el texto)
Anti Tráfico: Sí (se menciona el beneficio de elegir un rango horario para evitar el tráfico)
Evitar tráfico: Parcialmente (se menciona el beneficio de Anti Tráfico, pero no se menciona específicamente la idea de "evitar" el tráfico)


In [27]:
prompt = f"""
Determina si cada elemento en la lista de \
temas es un tema en el texto que está delimitado \
por tres comillas simples.

Proporciona tu respuesta como una lista con si, no o parcialmente \
en cada tema que si coincida, no coincida o parcialmente coincida.

En cada respuesta justifica entre paréntesis la coincidencia o no.

Lista de temas: {", ".join(bauTopicQuestion)}

Ficha técnica: '''{bauStory}'''
"""
response = get_completion(prompt)
print(response)

- Health: Sí (Se menciona el beneficio BAU Health)
- Lunch: Sí (Se mencionan los beneficios BAU Lunch y BAU Lunch Latam)
- Tráfico: No (No se menciona el tema del tráfico en el texto)
- Anti Tráfico: Sí (Se menciona el beneficio Anti Tráfico)
- Evitar tráfico: Parcialmente (Se menciona el beneficio Anti Tráfico que ofrece la posibilidad de elegir un rango horario para evitar el tráfico en el traslado al trabajo, pero no se menciona específicamente el tema de "evitar tráfico")


In [29]:
prompt = f"""
Determines whether each element in the list of topics\
is a topic in the text that is delimited by three single quotes.

Provide your answer as a list with yes, no, or partially \
for each topic that does match, does not match or partially matches.

For each answer explain the coincidence or not in parentheses.

Your answer must be in spanish.

List of topics: {", ".join(bauTopicQuestion)}

Story: '''{bauStory}'''
"""
response = get_completion(prompt)
print(response)

Health - Sí (Se menciona explícitamente el beneficio de salud para los colaboradores)
Lunch - Sí (Se menciona explícitamente el beneficio de almuerzo para los colaboradores)
Tráfico - No (No se menciona el tema del tráfico en el texto)
Anti Tráfico - Sí (Se menciona explícitamente el beneficio de evitar el tráfico en el traslado al trabajo)
evitar tráfico - No (No se menciona el tema del tráfico en el texto)
